In [1]:
%cd ..
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from melp import Detector
import melp
import math

import melp.clustering as clump

from melp.clustering.misc import*

#import melp.clustering.time_cluster as tclump
#import melp.clustering.spatial_cluster as sclump
#import melp.clustering.plots as clump_plt

plt.rcParams.update({'font.size': 16})

/home/erik/GitHub/melp
Welcome to JupyROOT 6.24/06


In [2]:
filename_trirec = "./testdata/trirec/trirec1.root"
filename_sorted = "./testdata/sorted/sorted1.root"
#filename_sorted_big = "/home/erik/GitLab_KIP/melp/root_files/big_run_data/sorted1.root"
filename_sorted_big = "./testdata/sorted/sorted1_big.root"

file = ROOT.TFile(filename_sorted)
ttree_mu3e = file.Get("mu3e")
ttree_mu3e_mc = file.Get("mu3e_mchits")
ttree_sensor = file.Get("alignment/sensors")
ttree_tiles = file.Get("alignment/tiles")

file_rec = ROOT.TFile(filename_trirec)
ttree_trirec = file_rec.Get("frames")
ttree_trirec_mc = file_rec.Get("frames_mc")
ttree_mu3e_segs = file_rec.Get("segs")

file_big = ROOT.TFile(filename_sorted_big)
ttree_mu3e_big = file_big.Get("mu3e")
ttree_mu3e_mc_big = file_big.Get("mu3e_mchits")
ttree_sensor_big = file_big.Get("alignment/sensors")
ttree_tiles_big = file_big.Get("alignment/tiles")

In [3]:
mu3e_detector = Detector.initFromROOT(filename_sorted)
#mu3e_detector = Detector.initFromSave("test_detector_file")

------------------------------
Detector information

Stats:
  - Tiles:  5824
    -> misal:  False
  - Pixel Modules:  2844
  - Loaded Runs (Tiles):  []
  - Loaded Runs (Pixel):  []
------------------------------


In [4]:
#select frame to analyze
frame_used = 30
ttree_mu3e.GetEntry(frame_used)

#build masks (and deconstruct them into arrays)
masks, __ = clump.masks.build_mask_around_cluster_master(ttree_mu3e, ttree_mu3e_mc, ttree_sensor, ttree_tiles, mu3e_detector, frame_used, mask_type = "big")

keys = []
values = []
for key in masks.keys():
    keys.append(key)
    values.append(masks[key])

In [5]:
clusters = clump.spatial_cluster.build_clusters_in_masks(ttree_mu3e, ttree_mu3e_mc, ttree_sensor, ttree_tiles,  mu3e_detector, frame_used, mask_type = "big", rec_type = None)

In [6]:
print(clusters[0])

Cluster(id=300460, frame_id=30, master_id=300460, master_primary=56141, hits=[ClusterHit(tile_id=300516, frame_id=30, primary=56141, time=6988.590792458939), ClusterHit(tile_id=300460, frame_id=30, primary=56141, time=6988.686382248762)])


In [7]:
clusters[0].get_tile_ids()

[300516, 300460]

In [8]:
clusters[0].get_primaries()

[56141, 56141]

In [9]:
clusters[0].get_times()

[6988.590792458939, 6988.686382248762]

In [10]:
clusters3 = clump.three_frame_cluster.build_clusters_in_masks_with_neighbours(ttree_mu3e, ttree_mu3e_mc, ttree_sensor, ttree_tiles,  mu3e_detector, frame_used, mask_type = "big", rec_type = None)

In [11]:
print(clusters3)

[Cluster(id=202890, frame_id=30, master_id=202890, master_primary=-48223, hits=[ClusterHit(tile_id=202890, frame_id=30, primary=-14701, time=-1), ClusterHit(tile_id=202778, frame_id=30, primary=-30340, time=6981.708047263179)]), Cluster(id=300460, frame_id=30, master_id=300460, master_primary=-48223, hits=[ClusterHit(tile_id=300460, frame_id=30, primary=56141, time=-1), ClusterHit(tile_id=300516, frame_id=30, primary=56141, time=6988.590792458939)]), Cluster(id=201644, frame_id=30, master_id=201644, master_primary=-48223, hits=[ClusterHit(tile_id=201644, frame_id=30, primary=-48223, time=-1), ClusterHit(tile_id=201588, frame_id=30, primary=-48223, time=7011.085776711903)]), Cluster(id=201855, frame_id=30, master_id=201855, master_primary=-48223, hits=[ClusterHit(tile_id=201855, frame_id=30, primary=4389, time=-1), ClusterHit(tile_id=201743, frame_id=30, primary=4389, time=6988.092201382493)]), Cluster(id=201855, frame_id=30, master_id=201855, master_primary=-48223, hits=[ClusterHit(til

In [12]:
clusters3[0]

Cluster(id=202890, frame_id=30, master_id=202890, master_primary=-48223, hits=[ClusterHit(tile_id=202890, frame_id=30, primary=-14701, time=-1), ClusterHit(tile_id=202778, frame_id=30, primary=-30340, time=6981.708047263179)])

In [13]:
for i in range(len(clusters3)):
    print(clusters3[i].get_tile_ids())

[202890, 202778]
[300460, 300516]
[201644, 201588]
[201855, 201743]
[201855, 201799]
[202890, 202779]
[202856, 202744]
[202856, 202800]
[300460]
[201855]
[202890]
[201045]
[202856]
[201644]


In [14]:
clusters3[6]

Cluster(id=202856, frame_id=30, master_id=202856, master_primary=-48223, hits=[ClusterHit(tile_id=202856, frame_id=30, primary=-52493, time=-1), ClusterHit(tile_id=202744, frame_id=30, primary=-52493, time=7034.646997290063)])

In [15]:
frac_mult_frame_cluster_hits, total_cluster_hits_counter, mult_frame_cluster_hits_counter = clump.three_frame_cluster.check_for_multiple_frame_clusters(ttree_mu3e_big, ttree_mu3e_mc_big, ttree_sensor_big, ttree_tiles_big, mu3e_detector, mask_type = "big", number_of_frames = 1000, rec_type = None)

Progress: 100 % of  1000  frames
Hits in cluster in different frame than master out of all hits in clusters:  4.253587808847462 %


In [16]:
double_hit_counter, total_cluster_hits_counter = clump.three_frame_cluster.check_for_mult_hit_tiles_diff_frame(ttree_mu3e_big, ttree_mu3e_mc_big, ttree_sensor_big, ttree_tiles_big, mu3e_detector, mask_type = "big", number_of_frames = 1000, rec_type = None)

Progress: 100 % of  1000  frames


In [18]:
hits_all_frames, hits_all_frames_counter_after = clump.three_frame_cluster.del_double_hits_in_3_frame_cluster(ttree_mu3e_big, ttree_mu3e_mc_big, ttree_sensor_big, ttree_tiles_big, mu3e_detector, mask_type="big", number_of_frames = 10000, rec_type = None)

Progress of building dictionary with all hits: 100 % of  10000  framesmes
cluster:  2 

hits:  2 

cluster:  2 

hits:  2 

cluster:  2 

hits:  2 

cluster:  2 

hits:  2 

cluster:  2 

hits:  2 

cluster:  2 

hits:  2 

cluster:  2 

hits:  2 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  3 

hits:  3 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

cluster:  4 

hits:  4 

c